<a href="https://colab.research.google.com/github/SuryaSurya1998/Visualization-Demographic-Analysis-of-USA/blob/main/Ball_Tree_Structure_Proximity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import modules
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file_list = drive.ListFile(
    {'q': "'1jGLe9hepFbAKQpDGNi0prk8Hc4mwyphX' in parents"}).GetList()  #use your own folder ID here

for f in file_list:
    # 3. Create & download by id.
    print('title: %s, id: %s' % (f['title'], f['id']))
    fname = f['title']
    print('downloading to {}'.format(fname))
    f_ = drive.CreateFile({'id': f['id']})
    f_.GetContentFile(fname)

title: Store_Demographics.docx, id: 1h-1Bm0bFL5a-JbHjGafT3vvjixsKWt1z
downloading to Store_Demographics.docx
title: Store_Table .parquet, id: 1NiFlEPS94W0dG-Zbv5yQ1g0l6GMcb_db
downloading to Store_Table .parquet
title: Price_table.parquet, id: 1BIL0xUGjoKiyApaj9JMNiQcRM5JYKmzs
downloading to Price_table.parquet


In [ ]:
import pandas as pd
import numpy as np

df_store = pd.read_parquet('Store_Table .parquet')

In [ ]:
#df_store.dropna(axis=0,how='any',subset=['MasterId','CompanyType'],inplace=True)

In [ ]:
df=df_store.loc[:,['StoreID','Latitude','Longitude']]
df.head()

,StoreID,Latitude,Longitude
0,8382,32.669012,-114.608225
1,8383,32.601897,-114.584900
2,8384,32.658674,-114.443464
3,8388,33.639345,-112.348586
4,8390,33.638848,-112.247623


In [ ]:
df1 =df.copy()
df1.tail()


,StoreID,Latitude,Longitude
60277,167561,40.628376,-73.737339
60278,167562,39.924476,-105.108263
60279,167563,38.920006,-76.975962
60280,167565,45.387095,-92.772886
60281,167566,32.723240,-116.962798


## Ball tree stucture

https://www.youtube.com/watch?v=fy40y3UFkNE

It is similar other tree structure in which it create a cluster of similar data point, on the top these we are considering our KNN so that order of complexity is reduced to log(n)

when a new querry came,

if the data point is in leaf node. it take distance between datapoint and cluster point, then take the shortest distance


or if data point is not in leaf node, somewhere in depth, it calclute distance between data point and leafnode centroid comes under that cluster, then take shortest distanace leaf node. Afterwards it calculate distance between datapoint and leaf node pick points, consider shortest distance

In [ ]:
import pandas as pd
import numpy as np

from sklearn.neighbors import BallTree
from sklearn.neighbors import KDTree
# from scipy import spatial
# from scipy.spatial import KDTree




# Setup Balltree using df as reference dataset
# Use Haversine calculate distance between points on the earth from lat/long
# haversine - https://pypi.org/project/haversine/ 

tree = BallTree(np.deg2rad(df[['Latitude', 'Longitude']].values),leaf_size =20, metric='haversine')

#tree = KDTree((df[['Latitude', 'Longitude']].values), metric='haversine')

In [ ]:

query_lats = df1['Latitude']
query_lons = df1['Longitude']


# use k = 5 for 5 closest neighbors

distances, indices = tree.query(np.deg2rad(np.c_[query_lats, query_lons]), k = 5)

r_km = 6371 # multiplier to convert to km (from unit distance)
for store, d, ind in zip(df1['StoreID'], distances, indices):
  print(f"StoreID {store} closest matches:")
  for i, index in enumerate(ind):
    print(f"\t{df1['StoreID'][index]} with distance {d[i]*r_km:.4f} km")
#print(d[10458])

Streaming output truncated to the last 5000 lines.
	25934 with distance 3.1289 km
	155484 with distance 3.9954 km
StoreID 161757 closest matches:
	161757 with distance 0.0000 km
	154684 with distance 4.9057 km
	165654 with distance 6.9558 km
	111406 with distance 7.0473 km
	135790 with distance 7.5129 km
StoreID 161758 closest matches:
	161758 with distance 0.0000 km
	151473 with distance 4.6197 km
	10806 with distance 4.6492 km
	129714 with distance 4.8332 km
	145694 with distance 6.4164 km
StoreID 161759 closest matches:
	161759 with distance 0.0000 km
	539 with distance 1.2374 km
	125127 with distance 1.3810 km
	112691 with distance 2.4549 km
	124926 with distance 4.0998 km
StoreID 161761 closest matches:
	161761 with distance 0.0000 km
	26009 with distance 0.4320 km
	124745 with distance 4.1080 km
	9078 with distance 4.2549 km
	144880 with distance 5.3958 km
StoreID 161763 closest matches:
	161763 with distance 0.0000 km
	47801 with distance 3.8644 km
	147711 with distance 6.8582 k

In [ ]:
np.shape(distances)

(60275, 5)

In [ ]:
np.shape(indices)



(60275, 5)

In [ ]:
distances[1][1]

0.0004298872579626904

In [ ]:
indices[8318]

array([ 8318,  5174,     0, 41541, 28298])

In [ ]:
for store, i, dis in zip(df1['StoreID'], indices, distances):
  print(store, i, dis)

Streaming output truncated to the last 5000 lines.
800549 [56569 57823 58014 57874 49952] [0.00000000e+00 5.98000043e-05 1.21037581e-04 1.45041796e-04
 2.60989353e-04]
800550 [56570 56348 57604 57645 56420] [0.00000000e+00 4.00683625e-05 6.86670388e-05 8.29800681e-05
 9.28350264e-05]
800551 [56571 57962 56850 58970 56685] [0.00000000e+00 7.63464999e-05 9.56444680e-05 2.33872261e-04
 2.62429439e-04]
800552 [56572 57461 57340 57458 57479] [0.00000000e+00 8.03324404e-05 8.34820880e-05 8.50537479e-05
 1.95527980e-04]
800553 [56573 50159 50166 56131 50162] [0.         0.00024396 0.00027528 0.00033307 0.00034607]
800554 [56574 56214 57993 58961 49972] [0.         0.00021733 0.00046568 0.00209593 0.00243807]
800555 [56575 49912 49751 49816 57970] [0.         0.00099972 0.00215756 0.00275384 0.00303041]
800556 [56576 58880 49736 49747 57958] [0.00000000e+00 8.61483768e-06 1.78147335e-04 2.78603883e-04
 3.06221450e-04]
800557 [56577 49633 58148 58152 57958] [0.         0.00017633 0.0001891  0.0